# Iterating through the list of authors

In [ ]:
import pickle
import re
import pandas as pd
from bs4 import BeautifulSoup
from parsing_page import parsing_author_page, parsing_article_page
import time
import requests
from os.path import exists
from os import makedirs


In [ ]:
from data_types import Author, AuthorsDB
from data_types import Publication, PublicationsDB
from data_types import Abstract, AbstractsDB
audb = AuthorsDB()
# audb.load()
pubdb = PublicationsDB()
# pubdb.load()
absdb = AbstractsDB()
# absdb.load()

In [ ]:
filename = "../data/mathnet_iam_authors_dict.pkl"
with open(filename,'rb') as inp:
    authors_dict = pickle.load(inp)

# Main functions for 

In [ ]:
def need_to_scrap(author, authors_db:AuthorsDB):
    interesting_list = ["Бондаренко", "Поляков", "Якобовский", "Четверушкин"]
    # interesting_list = ["Бондаренко", "Якобовский"]
    # interesting_list = ["Бондаренко"]
    for person in interesting_list:
        if person in author['name']:
            # we can demand some other conditions
            # if not authors_db.check_key(author["mn_id"]):
            if authors_db.check_key(author["mn_id"]):
                print(f"We have him {author} in authors_db")
                return True
            else:
                # print(f"We have him {author} in authors_db")
                print(f"We don't have him {author} in authors_db")
                return True
    return False 

In [ ]:
def save_html(data, mnid, name, status='write'):
    newpath = r'../data/'+mnid+'/'
    if not exists(newpath):
        makedirs(newpath)
    filename = "../data/"+mnid+"/"+name+".pkl"
    if status == 'write':        
        with open(filename,'wb') as outp:
            pickle.dump(data, outp, pickle.HIGHEST_PROTOCOL)

In [ ]:
def get_author_info(mnid, status="not_saving", web='offline'):
    print("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl")
    print(exists("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"))
    if exists("../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"):
        print("We read from file")
        filename = "../data/"+mnid+"/mn_author_"+mnid+"_page.pkl"
        with open(filename,'rb') as inp:
            author_page = pickle.load(inp)
    else:
        print(f"we need to request page for author mnid = {mnid}")
        if web=='offline':
            return None
        page_link = f"http://www.mathnet.ru/php/person.phtml?option_lang=rus&personid={mnid}"
        
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # page_link ="http://mi.mathnet.ru/+ {}"
        time.sleep(0.7)
        response = requests.get(page_link, headers=headers, timeout=None)
        if response.status_code == 200:
            author_page = response.content
        if status=="saving":
            save_html(author_page, mnid, "mn_author_"+mnid+"_page",status='write')
        author_page
                    
    soup = BeautifulSoup(author_page, 'html.parser')
    # TODO:
    # Need to check if this is real page and not for bot response
    return parsing_author_page(soup) 

In [ ]:
def get_pub_info(mn_id, mnlink, status="not_saving",web='offline'):
    print("../data/"+mn_id+"/"+mnlink+".pkl")
    print(exists("../data/"+mn_id+"/"+mnlink+".pkl"))
    if exists("../data/"+mn_id+"/"+mnlink+".pkl"):
        print("We read from file")
        filename = "../data/"+mn_id+"/"+mnlink+".pkl"
        with open(filename,'rb') as inp:
            pub_page = pickle.load(inp)
    else:
        print("we don't find that file")
        if web == 'offline':
            return None   
        print(f"we need to request page for pub_page mnlink = {mnlink}")
        page_link = "http://mi.mathnet.ru/" + mnlink
        print(page_link)
        # Chancge 'User-Agent' HEADERRRRR after some time of use !!!!
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # 
        time.sleep(0.5)
        response = requests.get(page_link, headers=headers, timeout=None)        
        print(response.status_code)        
        if response.status_code == 200:
            pub_page = response.content            
            print("we save file: " + mnlink)
            if status=="saving":
                save_html(pub_page, mn_id, mnlink,status='write')
        else:
            print(response.status_code)
            print(type(response.status_code))
            print("response.status_code != 200:")
            return "Stop"
        # pub_page        
    soup = BeautifulSoup(pub_page, 'html.parser')
    # if status != "working":
    #     return parsing_article_page(soup)
    
    return parsing_article_page(soup)

In [ ]:
audb.show()

In [ ]:
save_html_status = "saving"
status = "testing"
# safe
cur_web = 'offline'
#saveingDB = 'easy'
# -------------
# working
cur_web = 'online'
saveingDB = 'write'
for indx, item in authors_dict.items(): 
    request_problems = False
    if need_to_scrap(item, audb):                
        new_author = None
        new_author = Author(mn_id=item["mn_id"])
        print(item)
        # get through pubs list on author page
        author_pubs = None
        author_pubs = get_author_info(item["mn_id"], status=save_html_status,web=cur_web)
        if author_pubs is None:
            continue
        for pubs_indx, pubs_item in author_pubs.items():
            if 'mn_link' not in pubs_item:
                continue                
            print(pubs_indx, pubs_item)            
            if pubs_item["mn_link"] in pubdb.db.keys():
                # статья уже есть надо просто добавить к автору
                new_author.read_from_pubdb(pubs_item["mn_link"], audb)                
                continue
            
            if pubs_item["mn_link"] is not None:
                pub_page_info = None                                
                pub_page_info = get_pub_info(item["mn_id"],pubs_item["mn_link"],status=save_html_status,web=cur_web)
                # if len(pub_page_info)==1:
                # # if pub_page_info=="Stop":
                #     audb.save(f"we have len(pub_page_info)==1 \nsaving after author={item['mn_id']}, link={pubs_item['mn_link']}")
                #     pubdb.save()
                #     absdb.save()
                #     # request_problems = True
                #     continue
                
                    
                new_pub = Publication(pubs_item["mn_link"])
                new_abs = Abstract(pubs_item["mn_link"])
                
                if pub_page_info is not None:                    
                    new_author.update_author_info(pubs_item, pub_page_info)
                    new_pub.update_publication_info(pubs_item, pub_page_info)
                    new_abs.update_abstract_info(pubs_item, pub_page_info)                    
                    pubdb.update_data(new_pub.convert2dict())                            
                    absdb.update_data(new_abs.convert2dict())
                    for page_indx, page_item  in pub_page_info.items():
                        print(page_indx)
                        print(page_item)                              
        if request_problems == True:
            break
        print(new_author.convert2dict())
        audb.update_data(new_author.convert2dict())
        if status  == "testing":
            audb.show()
            pubdb.show()
            absdb.show()
        audb.save(status = saveingDB,text = f"we saving after {item['mn_id']} author")
        pubdb.save(status = saveingDB)
        absdb.save(status = saveingDB)

In [ ]:
audb.show()

In [ ]:
pubdb.show()

In [ ]:
absdb.show()